In [1]:
import numpy as np
import pandas as pd
import model
import nessra_genes_sets_generator as ng
import mygene 
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Matrix and Objective vector preparation

In [2]:
#------------- MATRIX and CLASS VECTOR
matrix = pd.read_csv("~/Desktop/expressionMatrix5.tsv", sep = "\t", header=0)
#class_vector = pd.read_csv("~/GoogleDrive/DataMiningLab/Model/objective.tsv", sep = "\t", header=0)
print("Matrix shape: {}".format(matrix.shape))
print(matrix.head(2))
print("")

Matrix shape: (60482, 103)
              Gene_ID  TCGA-DM-A28F-01A  TCGA-AD-6889-01A  TCGA-AF-A56L-01A  \
0   ENSG00000270112.3          0.000000          0.004395           0.00000   
1  ENSG00000167578.15          3.703513          2.473260           3.00915   

   TCGA-AY-4071-01A  TCGA-D5-5538-01A  TCGA-AZ-4616-01A  TCGA-DM-A0X9-01A  \
0          0.000000          0.000000          0.004094          0.008218   
1          5.262676          1.759248          3.576053          4.436359   

   TCGA-A6-A567-01A  TCGA-D5-5537-01A  ...  TCGA-G5-6233-01A  \
0          0.000000          0.000000  ...          0.004409   
1          3.441687          2.544031  ...          3.186217   

   TCGA-DY-A1DD-01A  TCGA-NH-A8F8-01A  TCGA-AZ-6607-01A  TCGA-5M-AAT6-01A  \
0          0.000000          0.000000          0.000000          0.000000   
1          4.406993          3.953229          1.713761          2.499009   

   TCGA-DM-A1DB-01A  TCGA-F5-6863-01A  TCGA-F4-6809-01A  TCGA-AF-3913-01A  \
0

In [3]:
# transposing matrix
matrix_t = matrix.T
matrix_t.columns = matrix_t.iloc[0]
matrix_t = matrix_t.drop("Gene_ID", axis=0)

# Resetting column names without the version of ENS
col_names = list(matrix_t.columns.values)
new_col_names = []
for name in col_names:
    temp_split = name.split(".")
    new_col_names.append(temp_split[0])
matrix_t.columns = new_col_names
print("Transpose matrix shape: {}".format(matrix_t.shape))
print("")

Transpose matrix shape: (102, 60482)



In [4]:
# class vector 
class_vector = pd.read_csv("~/Desktop/objective.tsv", sep = "\t", header=0)
print("Objective shape: {}".format(class_vector.shape))  

print(len([x for x in class_vector["over2years"] if x <1])/len(class_vector["over2years"]))
print(len(class_vector["over2years"]))

# generating barcode of samples
samples_bar = []
for name in list(matrix_t.index.values):
    samples_bar.append(name[:-4])
fil_for_class = class_vector.barcode.isin(samples_bar)
class_vector = class_vector[fil_for_class]
print("Objective shape after filter for matrix samples: {}".format(class_vector.shape))

# Final vector for classifying
train = class_vector["over2years"]
print("Final vector for classiftibg is a {}, first elements: {}".format(type(train), train[:5]))
print("")

Objective shape: (104, 3)
0.6634615384615384
104
Objective shape after filter for matrix samples: (102, 3)
Final vector for classiftibg is a <class 'pandas.core.series.Series'>, first elements: 0    1
1    1
2    0
3    0
4    1
Name: over2years, dtype: int64



# NESSRA

importing and preparing output genes

In [5]:
#----------------- NESSRA -----------------
print("----SVM on NESSRA----")
print("")
# importing files output
list_apc = [pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/160679_Hs.expansion', sep = ",", header=1),
            pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/160679_Hs.interactions', sep = ",", header=1)]
list_smad4 = [pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/160680_Hs.expansion', sep = ",", header=1),
            pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/160680_Hs.interactions', sep = ",", header=1)]
list_kras = [pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/160812_Hs.expansion', sep = ",", header=1),
            [None]]
list_braf = [pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/147717_Hs.expansion', sep = ",", header=1),
            [None]]

# importing tcode conversion matrix
tcode_mat = pd.read_csv('~/GoogleDrive/DataMiningLab/hgnc_filtered_anno.csv', sep = ",", header=0)

----SVM on NESSRA----



/home/riccardo/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# Set filter tresholds and max genes per set here!
tcode_symbol_dict = ng.tcodeSymbolDictGen(tcode_mat)

I'm creating tcode-to-Symbol dictionary, this will take a minute
Done, dictionary ready


In [48]:
# Constructing sets
freq_treshold = 0.8
n_genes_per_file = 250


set_apc = ng.setGen(list_apc[0], list_apc[1],freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=True)
set_smad4 = ng.setGen(list_smad4[0], list_smad4[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)
set_kras = ng.setGen(list_kras[0], list_kras[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)
set_braf = ng.setGen(list_braf[0], list_braf[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)

list_of_sets = [set_apc, set_smad4, set_braf, set_braf ]
print("Len of set apc: {}".format(len(set_apc)))
print("Len of set smad4: {}".format(len(set_smad4)))
print("Len of set kras: {}".format(len(set_kras)))
print("Len of set braf: {}".format(len(set_braf)))

Finished.
querying 1-8...done.
Finished.
querying 1-250...done.
Finished.
5 input query terms found dup hits:
	[('LY6G6E', 8), ('MIR21', 2), ('HNF1A-AS1', 2), ('GLIS3-AS1', 2), ('GUSBP6', 2)]
4 input query terms found no hit:
	['KIAA0430', 'ISPD', 'ALS2CR11', 'PARK2']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-147...done.
Finished.
1 input query terms found dup hits:
	[('HCG18', 8)]
5 input query terms found no hit:
	['CECR5', 'MARCH6', 'SSFA2', 'FAM49A', '2019-08-03 00:00:00']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
Len of set apc: 0
Len of set smad4: 8
Len of set kras: 266
Len of set braf: 164


In [49]:
ns_mat = model.select_table(matrix_t, list_of_sets)
print("Transpose matrix filtered shape: {}".format(ns_mat.shape))
print(ns_mat.head(2))

Transpose matrix filtered shape: (102, 150)
                 ENSG00000146083 ENSG00000022567 ENSG00000112511  \
TCGA-DM-A28F-01A         24.7199         5.12241         7.49493   
TCGA-AD-6889-01A         11.7048         2.86845         13.0324   

                 ENSG00000173276 ENSG00000184014 ENSG00000159082  \
TCGA-DM-A28F-01A         2.90286        0.509098         1.54334   
TCGA-AD-6889-01A         2.62479         1.96586        0.794179   

                 ENSG00000119522 ENSG00000101544 ENSG00000198642  \
TCGA-DM-A28F-01A         5.39781         3.56507         12.8879   
TCGA-AD-6889-01A         4.76901         6.51013         4.86383   

                 ENSG00000058272  ... ENSG00000120616 ENSG00000105339  \
TCGA-DM-A28F-01A         3.51156  ...         2.18634         2.71929   
TCGA-AD-6889-01A          2.3136  ...         1.85904         1.00252   

                 ENSG00000147383 ENSG00000109466 ENSG00000165030  \
TCGA-DM-A28F-01A         32.1805         2.55513     

# SVM analysis

splitting dataset, training model and evaluation

In [52]:
#------------ SVM ANALYSIS ------------

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(ns_mat.values, train.values, test_size=0.3,random_state=100) # 80% training and 20% test

In [53]:
print(X_train.shape)
print(len([x for x in y_train if x <1])/len(y_train))
print(X_test.shape)
print(len([x for x in y_test if x <1])/len(y_test))


(71, 150)
0.6619718309859155
(31, 150)
0.6451612903225806


In [54]:
# train the model
clf_ns = model.training(X_train, y_train)

y_pred = clf_ns.predict(X_test)


{'C': 10.0, 'gamma': 1.0}
fit_time 0.006757020950317383
score_time 0.021332184473673504
test_accuracy 0.660024154589372
test_f1 0.0
test_precision 0.0
test_recall 0.0


/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site

In [55]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])

In [58]:
print("--- Evaluating test set ---")
print("Test matrix shape: {}  \nTest class vector: {} \n".format(X_test.shape, len(y_test)))
print("Frequency filter used in NESSRA output: {}, final number of genes: {}". format(freq_treshold, X_test.shape[1]))
print("")
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))
print("")
print("Proportion of 0s in test y: {}".format(len([x for x in y_test if x <1])/len(y_test)))
print("Proportion of 0s in prediction y: {}".format(len([x for x in y_pred if x <1])/len(y_pred)))

--- Evaluating test set ---
Test matrix shape: (31, 150)  
Test class vector: 31 

Frequency filter used in NESSRA output: 0.8, final number of genes: 150

Accuracy: 0.6451612903225806
Precision: 0.0
Recall: 0.0

Proportion of 0s in test y: 0.6451612903225806
Proportion of 0s in prediction y: 1.0


/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


# Naive Bayes Model

In [59]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
g_model = GaussianNB()

# Train the model using the training sets
g_model.fit(X_train,y_train)



GaussianNB(priors=None, var_smoothing=1e-09)

In [60]:
#Predict Output
predicted= g_model.predict(X_test)
print("Proportion of 0s in test y: {}".format(len([x for x in y_test if x <1])/len(y_test)))
print("Proportion of 0s in prediction y: {}".format(len([x for x in predicted if x <1])/len(predicted)))

Proportion of 0s in test y: 0.6451612903225806
Proportion of 0s in prediction y: 0.45161290322580644


In [63]:
# Model Accuracy: how often is the classifier correct?
print(predicted)
print(y_test)
print("Accuracy:",metrics.accuracy_score(y_test, predicted))

[1 0 0 1 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0 1 0 0 1 1 0 1 0 1 1 1 0]
[0 0 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0]
Accuracy: 0.7419354838709677


In [71]:
print("Nessra treshold of {}, selected max {} genes per set:".format(freq_treshold, n_genes_per_file))
print("")
print("Len of ens id set apc: {}".format(len(set_apc)))
print("Len of ens id set smad4: {}".format(len(set_smad4)))
print("Len of ens id set kras: {}".format(len(set_kras)))
print("Len of ens id set braf: {}".format(len(set_braf)))
print("")
print("Transpose matrix intersection with sets shape: {}".format(ns_mat.shape))
print("")
print("---- Naive Bayes ----")
print("Proportion of 0s in test y: {}".format(len([x for x in y_test if x <1])/len(y_test)))
print("Proportion of 0s in prediction y: {}".format(len([x for x in predicted if x <1])/len(predicted)))
print("")
print(predicted)
print(y_test)
print("Accuracy:",metrics.accuracy_score(y_test, predicted))

Nessra treshold of 0.8, selected max 250 per set:

Len of ens id set apc: 0
Len of ens id set smad4: 8
Len of ens id set kras: 266
Len of ens id set braf: 164

Transpose matrix intersection with sets shape: (102, 150)

---- Naive Bayes ----
Proportion of 0s in test y: 0.6451612903225806
Proportion of 0s in prediction y: 0.45161290322580644

[1 0 0 1 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0 1 0 0 1 1 0 1 0 1 1 1 0]
[0 0 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0]
Accuracy: 0.7419354838709677


# Neural Network

In [68]:
# Import `Sequential` from `keras.models`
from keras.models import Sequential

# Import `Dense` from `keras.layers`
from keras.layers import Dense

n_input = X_train.shape[1]
n_hidden = int(np.ceil(n_input/3*2))
print("n input: {}, n perceptrons per hidden: {}".format(n_input, n_hidden))
# Initialize the constructor
nn_model = Sequential([
    
# Add an input layer 
Dense(n_input, activation='relu', input_shape=(n_input,)),
    
# Add three hidden layer 
Dense(n_hidden, activation='relu'),
Dense(n_hidden, activation='relu'),
Dense(n_hidden, activation='relu'),
# Add an output layer 
Dense(1, activation='sigmoid')
])


n input: 150, n perceptrons per hidden: 100


In [69]:
# Model compile
nn_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
                   
nn_model.fit(X_train, y_train,epochs=150, batch_size=1, verbose=1)

Epoch 1/150
71/71 [==============================] - 3s 44ms/step - loss: 5.4484 - acc: 0.6620
Epoch 2/150
71/71 [==============================] - 0s 7ms/step - loss: 5.4484 - acc: 0.6620
Epoch 3/150
71/71 [==============================] - 0s 6ms/step - loss: 5.4484 - acc: 0.6620
Epoch 4/150
71/71 [==============================] - 0s 7ms/step - loss: 5.4484 - acc: 0.6620
Epoch 5/150
71/71 [==============================] - 1s 10ms/step - loss: 5.4484 - acc: 0.6620
Epoch 6/150
71/71 [==============================] - 0s 6ms/step - loss: 5.4484 - acc: 0.6620
Epoch 7/150
71/71 [==============================] - 0s 7ms/step - loss: 5.4484 - acc: 0.6620
Epoch 8/150
71/71 [==============================] - 1s 10ms/step - loss: 5.4484 - acc: 0.6620
Epoch 9/150
71/71 [==============================] - 1s 8ms/step - loss: 5.4484 - acc: 0.6620
Epoch 10/150
71/71 [==============================] - 1s 7ms/step - loss: 5.4484 - acc: 0.6620
Epoch 11/150
71/71 [==============================] - 1s

71/71 [==============================] - 0s 6ms/step - loss: 5.4484 - acc: 0.6620
Epoch 88/150
71/71 [==============================] - 0s 6ms/step - loss: 5.4484 - acc: 0.6620
Epoch 89/150
71/71 [==============================] - 0s 7ms/step - loss: 5.4484 - acc: 0.6620
Epoch 90/150
71/71 [==============================] - 0s 6ms/step - loss: 5.4484 - acc: 0.6620
Epoch 91/150
71/71 [==============================] - 0s 7ms/step - loss: 5.4484 - acc: 0.6620
Epoch 92/150
71/71 [==============================] - 0s 6ms/step - loss: 5.4484 - acc: 0.6620
Epoch 93/150
71/71 [==============================] - 0s 7ms/step - loss: 5.4484 - acc: 0.6620
Epoch 94/150
71/71 [==============================] - 0s 6ms/step - loss: 5.4484 - acc: 0.6620
Epoch 95/150
71/71 [==============================] - 0s 7ms/step - loss: 5.4484 - acc: 0.6620
Epoch 96/150
71/71 [==============================] - 0s 6ms/step - loss: 5.4484 - acc: 0.6620
Epoch 97/150
71/71 [==============================] - 0s 6ms/st

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/riccardo/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-69-1a767601116c>", line 6, in <module>
    nn_model.fit(X_train, y_train,epochs=150, batch_size=1, verbose=1)
  File "/home/riccardo/miniconda3/lib/python3.7/site-packages/keras/engine/training.py", line 1039, in fit
    validation_steps=validation_steps)
  File "/home/riccardo/miniconda3/lib/python3.7/site-packages/keras/engine/training_arrays.py", line 199, in fit_loop
    outs = f(ins_batch)
  File "/home/riccardo/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py", line 2715, in __call__
    return self._call(inputs)
  File "/home/riccardo/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py", line 2675, in _call
    fetched = self._callable_fn(*array_vals)
  File "/home/riccardo/miniconda3/lib/python3.7/site

KeyboardInterrupt: 

In [66]:
y_pred = nn_model.predict(X_test)
y_pred = [int(x) for x in y_pred]
print(y_pred)
print("Proportion of 0s in test y: {}".format(len([x for x in y_test if x <1])/len(y_test)))
print("Proportion of 0s in prediction y: {}".format(len([x for x in y_pred if x <1])/len(y_pred)))


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Proportion of 0s in test y: 0.6451612903225806
Proportion of 0s in prediction y: 0.967741935483871


In [67]:
# Evaluation
score = nn_model.evaluate(X_test, y_test,verbose=1)

print(score)

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

31/31 [==============================] - 0s 14ms/step
[3.952394485473633, 0.5161290168762207]
Accuracy: 0.6129032258064516


# Symbol gene printing

In [111]:
# Save genes as SYMBOL

ens_gene_list = list(ns_mat.columns.values)
mg = mygene.MyGeneInfo()
symbol_genes = mg.querymany(ens_gene_list, scopes='ensembl.gene', fields='symbol', species='human')
gene_list = []
for el in symbol_genes:
    if "symbol" in el:
        gene_list.append(el["symbol"])
print("len of ens genes: {}, len of symbol genes: {}".format(len(ens_gene_list), len(gene_list)))
print(', '.join(gene_list))

querying 1-482...done.
Finished.
len of ens genes: 482, len of symbol genes: 482
RNF44, PTPN4, SLC45A4, PHF1, DGKD, NOP16, ZBTB21, TAF9B, DENND5A, URB1-AS1, CREM, USP9X, CCDC90B, SYNJ1, TTLL1, ZNF622, DENND1A, ANKRD42, POMC, SLC39A10, TUBA1C, SCYL2, LSM14A, ADNP2, KLHL9, KMT2E, PPP1R12A, SDAD1P1, CAMKK2, CUL1, GPCPD1, PLAG1, MIDN, PCBP1-AS1, MPC1, CWC25, PXYLP1, IRGQ, PI4KA, ATF7IP, NUP58, TTC12, LAMTOR2, FYN, ST13P6, NUP98, VWA7, SMAP2, TMEM80, GAB2, ZHX2, USP34, CAMK1D, ANKRD11, FAM160A2, ZNF266, KIAA1109, MBD6, ENTPD4, GABPA, ALKBH3, TMEM216, ZFX, CHKB, PSMA7, BCL9, FOXO1, CRTC2, ZKSCAN7, AK4, ORMDL1, TRAPPC8, ARF4, CCNL1, PHF20L1, ZNF264, LUC7L2, HSP90AB1, METTL21A, DDX50, TTC39C, SLCO4A1, RBM45, JMJD6, NFXL1, TSTA3, SMDT1, FOXN3, ORMDL2, AHCTF1, RFX2, SLC50A1, KLHL18, NDUFC1, TNFAIP8, ZBTB17, CREBRF, NFAT5, RSPO1, IFRD1, SLC12A6, FAM122A, PGS1, SF1, NEK9, FAM126B, PSME4, PPP1R3F, KCNAB2, DFFA, XPR1, COA6, RLIM, HINT2, RMDN1, BCAP31, FNIP1, GSTP1, HMGB1, KDM5C, SPAG4, POLR2J4, DHCR

# Using single NESSRA FILES

In [100]:
# Constructing sets
freq_treshold = 0.3
n_genes_per_file = 250


#set_apc = ng.setGen(list_apc[0], list_apc[1],freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=True)
set_smad4 = ng.setGen(list_smad4[0], list_smad4[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)
#set_kras = ng.setGen(list_kras[0], list_kras[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)
set_braf = ng.setGen(list_braf[0], list_braf[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)

#list_of_sets = [set_apc, set_smad4, set_braf, set_braf ]
list_of_sets = [set_smad4, set_braf]
#print("Len of set apc: {}".format(len(set_apc)))
print("Len of set smad4: {}".format(len(set_smad4)))
#print("Len of set kras: {}".format(len(set_kras)))
print("Len of set braf: {}".format(len(set_braf)))

querying 1-250...done.
Finished.
10 input query terms found dup hits:
	[('SDAD1P1', 2), ('MIR4523', 2), ('CHCHD2P6', 2), ('MIR99AHG', 2), ('HCG25', 8), ('URB1-AS1', 2), ('
3 input query terms found no hit:
	['LINC00441', 'PROSC', 'ZCCHC11']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-250...done.
Finished.
4 input query terms found dup hits:
	[('HCG18', 8), ('MIR4727', 2), ('PCBP1-AS1', 2), ('POLR2J4', 2)]
7 input query terms found no hit:
	['CECR5', 'MARCH6', 'SSFA2', 'FAM49A', '2019-08-03 00:00:00', 'DIRC2', '2019-08-02 00:00:00']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
Len of set smad4: 279
Len of set braf: 275


In [109]:
ns_mat = model.select_table(matrix_t, list_of_sets)
print("Transpose matrix filtered shape: {}".format(ns_mat.shape))
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(ns_mat.values, train.values, test_size=0.3,random_state=100) # 80% training and 20% test


Transpose matrix filtered shape: (102, 482)


### SVM

In [102]:
# train the model
clf_ns = model.training(X_train, y_train)

y_pred = clf_ns.predict(X_test)
print("")
print("Nessra treshold of {}, selected max {} genes per set:".format(freq_treshold, n_genes_per_file))
print("")
#print("Len of ens id set apc: {}".format(len(set_apc)))
print("Len of ens id set smad4: {}".format(len(set_smad4)))
#print("Len of ens id set kras: {}".format(len(set_kras)))
print("Len of ens id set braf: {}".format(len(set_braf)))
print("")
print("Transpose matrix intersection with sets shape: {}".format(ns_mat.shape))
print("")
print("---- Naive Bayes ----")
print("Proportion of 0s in test y: {}".format(len([x for x in y_test if x <1])/len(y_test)))
print("Proportion of 0s in prediction y: {}".format(len([x for x in y_pred if x <1])/len(y_pred)))
print("")
print(y_test)
print(y_pred)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

{'C': 10.0, 'gamma': 1.0}
fit_time 0.008128643035888672
score_time 0.011251370112101236
test_accuracy 0.6618357487922706
test_f1 0.0
test_precision 0.0
test_recall 0.0

Nessra treshold of 0.3, selected max 250 genes per set:

Len of ens id set smad4: 279
Len of ens id set braf: 275

Transpose matrix intersection with sets shape: (102, 482)

---- Naive Bayes ----
Proportion of 0s in test y: 0.6451612903225806
Proportion of 0s in prediction y: 1.0

[0 0 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy: 0.6451612903225806


/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site

### Bayes

In [110]:
#Create a Gaussian Classifier
g_model = GaussianNB()

# Train the model using the training sets
g_model.fit(X_train,y_train)

predicted= g_model.predict(X_test)

print("Nessra treshold of {}, selected max {} genes per set:".format(freq_treshold, n_genes_per_file))
print("")
#print("Len of ens id set apc: {}".format(len(set_apc)))
print("Len of ens id set smad4: {}".format(len(set_smad4)))
#print("Len of ens id set kras: {}".format(len(set_kras)))
print("Len of ens id set braf: {}".format(len(set_braf)))
print("")
print("Transpose matrix intersection with sets shape: {}".format(ns_mat.shape))
print("")
print("---- Naive Bayes ----")
print("Proportion of 0s in test y: {}".format(len([x for x in y_test if x <1])/len(y_test)))
print("Proportion of 0s in prediction y: {}".format(len([x for x in predicted if x <1])/len(predicted)))
print("")
print(y_test)
print(predicted)
print("Accuracy:",metrics.accuracy_score(y_test, predicted))


Nessra treshold of 0.3, selected max 250 genes per set:

Len of ens id set smad4: 279
Len of ens id set braf: 275

Transpose matrix intersection with sets shape: (102, 482)

---- Naive Bayes ----
Proportion of 0s in test y: 0.6451612903225806
Proportion of 0s in prediction y: 0.5483870967741935

[0 0 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0]
[0 0 0 1 1 0 0 0 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 1 0 0 0 1 0]
Accuracy: 0.8387096774193549
